 Copyright 2024 shins777@gmail.com

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

In [1]:
%pip install --upgrade --quiet google-cloud-texttospeech google-cloud-translate google-cloud-speech google-cloud-aiplatform playsound

Note: you may need to restart the kernel to use updated packages.


In [6]:
# https://stackoverflow.com/questions/73268630/error-could-not-build-wheels-for-pyaudio-which-is-required-to-install-pyprojec
# For linux 

!echo Y | sudo apt install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 0 to remove and 9 not upgraded.
Need to get 596 kB of archives.
After this operation, 4,350 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libasound2-dev amd64 1.2.6.1-1ubuntu1 [110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack0 amd64 1:0.125.0-3build2 [93.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjack-dev amd64 1:0.125.0-3build2 [206 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:5 http:

In [7]:
%pip install --upgrade --quiet pyaudio

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
import sys
import threading

directory = os.getcwd()
# Append sys path to refer utils.
sys.path.append(directory+"/src")

import listen
from listen import MicrophoneStream
from process import VoiceTranslation
vo_tran = VoiceTranslation()

In [9]:
%load_ext autoreload
%autoreload 2

In [10]:
source_lang_options = (['한국어','ko-KR'],
            ['영어','en-US'],
            ['일본어','ja-JP'],
            ['베트남어','vi-VN'],
            ['말레이시아어','ms-MY'],
            ['인도네시아어','id-ID'],
            ['필리핀어','fil-PH'])

target_lang_options = (['한국어','ko-KR'],
            ['영어','en-US'],
            ['일본어','ja-JP'],
            ['베트남어','vi-VN'],
            ['말레이시아어','ms-MY'],
            ['인도네시아어','id-ID'],
            ['캄보디아','km-KH'],
            ['필리핀어','fil-PH'])

In [11]:
class ListenThread(threading.Thread):
    def __init__(self, stop_event):
        super(ListenThread, self).__init__()
        self.stop_event = stop_event    

    def run(self):
        listen.process(self.stop_event)

def looper_control(state, stop_event):
    if state == 'start':
        t = ListenThread(stop_event=stop_event)
        t.start()
    elif state == 'stop':
        stop_event.set()

In [ ]:
MicrophoneStream.receive_q.queue.clear()

stop_thread_event = threading.Event()
looper_control('start', stop_thread_event)

translated_dict = vo_tran.interpret(source_lang_code = source_lang_options[0][1],
                                    target_lang_code = target_lang_options[1][1],
                                    lang_options = target_lang_options
                                    )

from playsound import playsound
playsound('output.mp3')  

looper_control('stop', stop_thread_event)

print(translated_dict)